In [1]:
import transformers
from datasets import load_dataset, load_metric
import pandas as pd

c:\Users\love4\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
news = load_dataset("csv", data_files="./data/news_collection.csv")

Using custom data configuration default-8807e4642325d1fa
Found cached dataset csv (C:/Users/love4/.cache/huggingface/datasets/csv/default-8807e4642325d1fa/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|██████████| 1/1 [00:00<00:00, 125.44it/s]


In [3]:
datasets_train_validation = news["train"].train_test_split(test_size=3000)

news["train"] = datasets_train_validation["train"].shuffle()
news["validation"] = datasets_train_validation["test"].shuffle()

In [4]:
import nltk
nltk.download('punkt')
import string

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("yihsuan/mt5_chinese_small")

model = AutoModelForSeq2SeqLM.from_pretrained("yihsuan/mt5_chinese_small")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\love4\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def preprocess_data(examples):
  model_inputs = tokenizer(examples["desc"])

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"])

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [6]:
tokenized_datasets = news.map(preprocess_data, batched=True)

  0%|          | 0/45 [00:00<?, ?ba/s]c:\Users\love4\anaconda3\envs\pytorch\lib\site-packages\transformers\tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
 67%|██████▋   | 2/3 [00:00<00:00, 12.01ba/s]


In [7]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [8]:
batch_size = 4
model_name = "mt5-base-news-title-generation"
model_dir = f"./Models/{model_name}"

args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=500,
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="steps",
    save_steps=1000,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

In [9]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [10]:
metric = load_metric("rouge")

C:\Users\love4\AppData\Local\Temp\ipykernel_17080\3314070117.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [11]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) 
                      for label in decoded_labels]
    
    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [12]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained("yihsuan/mt5_chinese_small")

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

loading configuration file config.json from cache at C:\Users\love4/.cache\huggingface\hub\models--yihsuan--mt5_chinese_small\snapshots\338c78a9a4e6ff81d4e613a7b742a99b864b8f28\config.json
Model config MT5Config {
  "_name_or_path": "yihsuan/mt5_chinese_small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 250100
}

loadi

In [13]:
%load_ext tensorboard
%tensorboard --logdir '{model_dir}'/runs

Reusing TensorBoard on port 6006 (pid 19936), started 0:21:14 ago. (Use '!kill 19936' to kill it.)

In [14]:
trainer.train()

loading configuration file config.json from cache at C:\Users\love4/.cache\huggingface\hub\models--yihsuan--mt5_chinese_small\snapshots\338c78a9a4e6ff81d4e613a7b742a99b864b8f28\config.json
Model config MT5Config {
  "_name_or_path": "yihsuan/mt5_chinese_small",
  "architectures": [
    "MT5ForConditionalGeneration"
  ],
  "d_ff": 1024,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "mt5",
  "num_decoder_layers": 8,
  "num_heads": 6,
  "num_layers": 8,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "tokenizer_class": "T5Tokenizer",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "use_cache": true,
  "vocab_size": 250100
}

loadi

{'loss': 3.392, 'learning_rate': 3.879018382685864e-05, 'epoch': 0.05}
